In [ ]:
#!pip install xgboost

In [36]:
!pip install selenium beautifulsoup4 webdriver-manager


Defaulting to user installation because normal site-packages is not writeable
  Using cached selenium-4.25.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)
  Using cached trio-0.26.2-py3-none-any.whl.metadata (8.6 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
Using cached selenium-4.25.0-py3-none-any.whl (9.7 MB)
Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl (27 kB)
Using cached trio-0.26.2-py3-none-any.whl (475 kB)
Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

def obtener_imagenes_pinterest(busqueda, max_images=10):
    # Inicializa las opciones del navegador
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Modo sin cabeza, para que no abra el navegador
    options.add_argument("--disable-gpu")  # Deshabilita la GPU para evitar algunos errores
    options.add_argument("--no-sandbox")  # Evitar problemas en contenedores
    options.add_argument("--disable-dev-shm-usage")  # Solución de problemas de memoria compartida en Linux

    # Inicializa el driver de Chrome en modo headless
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Construir la URL de búsqueda en Pinterest
    busqueda_url = f"https://www.pinterest.com/search/pins/?q={busqueda.replace(' ', '%20')}"

    # Cargar la página
    driver.get(busqueda_url)
    time.sleep(5)  # Esperar a que cargue la página completamente

    # Desplazarse hacia abajo para cargar más imágenes (opcional)
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

    # Usar BeautifulSoup para analizar el HTML
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extraer las URLs de las imágenes
    imagenes = []
    for img_tag in soup.find_all('img', {'src': True}):
        src = img_tag['src']
        if src and 'pinimg.com' in src:  # Filtrar URLs de Pinterest
            imagenes.append(src)
            if len(imagenes) >= max_images:
                break

    driver.quit()  # Cerrar el navegador
    return imagenes




In [45]:
# Ejemplo de uso
imagenes_encontradas = obtener_imagenes_pinterest('shoes', max_images=7)
for idx, img_url in enumerate(imagenes_encontradas):
    print(f"Imagen {idx + 1}: {img_url}")

Imagen 1: https://i.pinimg.com/236x/fa/0d/07/fa0d073eb1fec2ec336e3a71adc95da1.jpg
Imagen 2: https://i.pinimg.com/236x/71/12/ca/7112ca6574ee056db43b8c861bd08b3d.jpg
Imagen 3: https://i.pinimg.com/236x/10/9c/c0/109cc068a74c61f29d7733549c197029.jpg
Imagen 4: https://i.pinimg.com/236x/dc/41/7e/dc417e0aabde7cad29f616f7fe10abe7.jpg
Imagen 5: https://i.pinimg.com/236x/8b/29/79/8b2979d19fb837541a0a57f8424045e0.jpg
Imagen 6: https://i.pinimg.com/236x/ec/a6/83/eca683899c09c368bef4c37e55628659.jpg
Imagen 7: https://i.pinimg.com/videos/thumbnails/originals/ac/9e/59/ac9e59c1a3677b5ea68392efebaecd92.0000000.jpg


In [16]:
#import spacy
from sklearn.model_selection import train_test_split
import pandas as pd
from scipy.sparse import hstack
import joblib
import xgboost as xgb
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier
import gc
import scipy.sparse as sp
from sklearn.neighbors import KNeighborsClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [17]:
# Descargar stopwords si es necesario
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Odiseo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Odiseo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Odiseo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:
df = pd.read_excel('Data_original.csv')

In [29]:
df.head()

,ropa,genero,estatura,contextura Fisica,ojos,piel,personalidad,ocasion
0,camisa,"hombre,mujer","alto,medio,alta","corpulento,delgado,atletico,robusto,corpulenta...","verde,azules,marrones,grises,avellana","negra,blanca,morena,moreno","seguro, confiado, audaz,tranquilo, relajado, i...","Reunion,Entrevista,Cenas formales,Cenas de neg..."
1,bermuda,"hombre,mujer","alta,alto,mediano,baja","Delgado, normal, atlético,corpulento","verde,azules,marrones,grises,avellana,amarillo...","negra,blanca,morena,moreno,clara","casual, relajado, práctico,extrovertida","Informales ,casual,playa,casa,vacaciones"
2,camiseta,"hombre,mujer","baja, media, alta","delgado, atlético, corpulento,delgada, atlétic...",NaN,"clara, morena, oscura","casual, relajado, deportivo, extrovertido, rel...","salidas,actividad deportiva,paseos,cine,reunio..."
3,chaquetas,"hombre,mujer","baja, media, alta","delgado, atlético, corpulento,delgada, atlétic...","marron, verde, azul, gris","clara, morena, oscura","seguro, sofisticado, extrovertido, relajado,se...","salida nocturna,evento fomal,reunion al aire l..."
4,falda,mujer,"baja, media, alta","delgada, atlética, corpulenta,gorda","marron, verde, azul, gris,rojo,amarillo","clara, morena, oscura","Extrovertida, sofisticada, creativa, segura","cenas formales,reuniones,fiestas,evento de verano"


In [20]:
def split_column_and_expand(df, column_to_split):
    """
    Divide una columna de un DataFrame en múltiples filas y expande las otras columnas.

    Args:
        df: El DataFrame de entrada.
        column_to_split: El nombre de la columna a dividir.

    Returns:
        Un DataFrame con la columna dividida en nuevas filas.
    """
    # Verificar si la columna ya es de tipo string
    if df[column_to_split].dtype != 'object':
        df[column_to_split] = df[column_to_split].astype(str)

    # Convertir los valores a minúsculas y dividir por comas en una sola operación
    df[column_to_split] = df[column_to_split].str.lower().str.split(',')

    # Explode para dividir las listas en nuevas filas
    df_expanded = df.explode(column_to_split, ignore_index=True)

    # Eliminar espacios en una sola operación usando applymap para evitar copias adicionales
    df_expanded[column_to_split] = df_expanded[column_to_split].apply(lambda x: x.strip() if isinstance(x, str) else x)

    return df_expanded


In [21]:
def expand_all_columns(df):
    """
    Expande todas las columnas de un DataFrame que contengan valores separados por comas.

    Args:
        df: El DataFrame de entrada.

    Returns:
        Un nuevo DataFrame con todas las columnas expandidas.
    """
    for col in df.columns:
        if df[col].apply(lambda x: isinstance(x, str) and ',' in x).any():
            df = split_column_and_expand(df, col)

    return df

In [22]:
def preprocess_and_vectorize(df, vectorizers=None):
    if vectorizers is None:
        vectorizers = {}

    # Corregir el filtrado de filas vacías de manera más robusta
    df = df[df.apply(lambda row: any(row.str.strip()), axis=1)]
    df.reset_index(drop=True, inplace=True)

    X_text_vectorized = None

    # Procesar columna por columna
    for col in df.columns:
        if col in vectorizers:
            # Usar vectorizador existente
            vectorizer = vectorizers[col]
            X_col_vectorized = vectorizer.transform(df[col])
        else:
            # Crear nuevo vectorizador si no existe
            vectorizer = TfidfVectorizer(stop_words=stopwords.words('spanish'), min_df=1)
            vectorizer.fit(df[col])
            X_col_vectorized = vectorizer.transform(df[col])
            vectorizers[col] = vectorizer

        # Concatenar las matrices en una sola (si es la primera, inicializarla)
        if X_text_vectorized is None:
            X_text_vectorized = X_col_vectorized
        else:
            X_text_vectorized = sp.hstack([X_text_vectorized, X_col_vectorized])

        gc.collect()  # Liberar memoria después de cada columna

    return X_text_vectorized,vectorizers

In [23]:
df.columns

Index(['ropa', 'genero', 'estatura', 'contextura Fisica', 'ojos', 'piel',
       'personalidad', 'ocasion'],
      dtype='object')

In [30]:
df_expanded = expand_all_columns(df)

In [33]:
df_expanded['ropa'].unique()

array(['camisa', 'bermuda', 'camiseta', 'chaquetas', 'falda',
       'manga_larga', 'pantalon', 'sombrero', 'vestido', 'zapato',
       'pulsera masculina', 'reloj masculino'], dtype=object)

In [31]:
len(df_expanded)

1680024

In [ ]:
# Encuentra el tamaño de la categoría más grande
max_size = df_expanded.groupby('ropa').size().max()

# Aplica sobremuestreo a cada categoría
df_expanded = df_expanded.groupby('ropa').apply(lambda x: x.sample(max_size, replace=True)).reset_index(drop=True)

In [ ]:
# Encuentra el tamaño de la categoría más grande
max_size = df_expanded.groupby('genero').size().max()

# Aplica sobremuestreo a cada categoría
df_expanded = df_expanded.groupby('genero').apply(lambda x: x.sample(max_size, replace=True)).reset_index(drop=True)

In [32]:
len(df_expanded)

1680024

In [ ]:
for col in df_expanded.columns:
    print(col,len(df_expanded[col].unique()))

ropa 10
genero 2
estatura 6
contextura Fisica 14
ojos 13
piel 7
personalidad 29
ocasion 33


In [ ]:
print(df_expanded['ropa'].value_counts())

ropa
pantalon       1269063
sombrero       1267864
bermuda        1267820
manga_larga    1266931
zapato         1266886
chaquetas      1266383
camiseta       1266309
camisa         1266208
falda          1014362
vestido        1013174
Name: count, dtype: int64


In [ ]:
labels_ropa= {categoria: i for i, categoria in enumerate(df_expanded['ropa'].unique())}

In [ ]:
X = df_expanded.drop(columns=['ropa'])
y = df_expanded['ropa'].map(labels_ropa)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
x_train.isna().sum()

,0
genero,0
estatura,0
contextura Fisica,0
ojos,1012707
piel,0
personalidad,0
ocasion,2026332


In [ ]:
x_train.fillna('desconocido', inplace=True)
x_test.fillna('desconocido', inplace=True)

In [ ]:
x_train.isna().sum()

,0
genero,0
estatura,0
contextura Fisica,0
ojos,0
piel,0
personalidad,0
ocasion,0


In [ ]:
x_train.shape[1]

7

In [ ]:
train,vectorizador = preprocess_and_vectorize(x_train)

In [ ]:
joblib.dump(vectorizador, 'vectorizador.pkl')
joblib.dump(train, 'train.pkl')

['train.pkl']

In [ ]:
x_train.shape[1]

7

In [ ]:
#train = joblib.load('/content/train.pkl')
#vectorizador = joblib.load('/content/vectorizador.pkl')

In [ ]:

model_xgb_multi = xgb.XGBClassifier(
    objective='multi:softmax',  # Cambiar a 'multi:softmax' o 'multi:softprob'
    num_class=10,                # Número de clases (0, 1, 2, 3, 4, 5)
    n_estimators=200,
    max_depth=6,
    learning_rate=0.08,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1,
    reg_alpha=0,
    random_state=42
)


In [ ]:
model_xgb_multi.fit(train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.08, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None, num_class=10,
              num_parallel_tree=None, ...)

In [ ]:
joblib.dump(model_xgb_multi, 'model_xgb_multi.pkl')

['model_xgb_multi.pkl']

In [ ]:
#model_lgb_multi = joblib.load('/content/model_lgb_multi.pkl')
#model_xgb_multi = joblib.load('/content/model_xgb_multi.pkl')
#model_rf = joblib.load('/content/model_rf.pkl')
#vectorizador = joblib.load('/content/vectorizador.pkl')

In [ ]:
print("Hola")

Hola


In [ ]:
test,_ = preprocess_and_vectorize(x_test,vectorizador)

In [ ]:
predictions_xgb_multi = model_xgb_multi.predict(test)

In [ ]:
accuracy = accuracy_score(y_test, predictions_xgb_multi) # Use iloc to access the column by index in the DataFrame
print(f"Precision: {accuracy*100}")

Precision: 98.19013563501849


In [ ]:
print("todo bien")

todo bien
